In [1]:
import sys
import numpy as np
import pickle
import os
import random
import argparse
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Conv2D, Dropout, GlobalAveragePooling2D,
                                    MaxPooling2D, Activation, Dense, Layer)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.callbacks import LearningRateScheduler, Callback
from tensorflow.keras import backend as K 
sys.path.append('../python_scripts/')
import analysis, datasets

# Function to make models
def init_all_cnn_c(seed: int):
    
    model = Sequential()
    # model.add(Dropout(0.2, input_shape=x_train.shape[1:])) #input shape from keras cifar10 example
    model.add(Conv2D(96, (3, 3), input_shape=(32, 32, 3), padding='same',
                        kernel_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), 
                        bias_initializer='zeros', activation='relu'))
    model.add(Conv2D(96, (3, 3), padding='same', kernel_regularizer=l2(1e-5),
                        kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
    model.add(Conv2D(96, (3, 3), strides=2, padding='same', 
                        kernel_regularizer=l2(1e-5), bias_initializer='zeros', activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5),
                        kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
    model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=l2(1e-5),
                        kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
    model.add(Conv2D(192, (3, 3), strides=2, padding='same',kernel_regularizer=l2(1e-5),
                        kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Conv2D(192, (3, 3), padding='valid', kernel_regularizer=l2(1e-5),
                        kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
    model.add(Conv2D(192, (1, 1), padding='valid', kernel_regularizer=l2(1e-5),
                        kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
    model.add(Conv2D(10, (1, 1), padding='valid', kernel_regularizer=l2(1e-5),
                        kernel_initializer=he_normal(seed), bias_initializer='zeros', activation='relu'))
    model.add(GlobalAveragePooling2D())
    model.add(Activation('softmax'))
    
    model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9, clipnorm=500),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

    return model

# Make a dummy function
def init_dummy(seed: int):

    model = Sequential()
    model.add(Conv2D(10, (3, 3), input_shape=(32, 32, 3), padding='same',
                        kernel_regularizer=l2(1e-5), kernel_initializer=he_normal(seed), 
                        bias_initializer='zeros', activation='relu'))
    model.add(GlobalAveragePooling2D())
    model.add(Activation('softmax'))
    
    model.compile(optimizer=SGD(learning_rate=0.01, momentum=0.9, clipnorm=500),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

    return model

# Scheduler callback
def scheduler(epoch, lr):
    if (epoch == 200 or epoch == 250 or epoch == 300):
        return lr * 0.1
    return lr

LR_Callback = LearningRateScheduler(scheduler)

# Trajectory callback
class Trajectory_Callback(Callback):
    '''
    Pre: Must define i, x_predict
    '''
    def on_epoch_end(self, epoch, logs=None):
        layer_arr = [7]
        if epoch in [0, 1, 2, 3, 4, 5,
                     6, 7, 8, 9,
                     49, 99, 149, 199, 249, 299, 349]:
            print('\n\nSnapshot instance', str(i), 'at epoch', str(int(epoch)+1))
            acts = analysis.get_acts(self.model, layer_arr, x_predict, cocktail_blank=False)
            np.save('../outputs/representations/acts/ten_by_ten/w'+str(w)+'s'+str(s)+'e'+str(epoch)+'.npy', acts)
            print('\n')


# Cut off training if local minimum hit  
class Early_Abort_Callback(Callback):
    '''
    Pre: abort is set to False at the beginning of each training instance
    '''
    def on_epoch_end(self, epoch, logs=None):
        global abort
        if (epoch > 10 and logs.get('accuracy') <= 0.11 or
                epoch == 70 and logs.get('accuracy') < 1.0):
            abort = True
            print('Acc:', logs.get('accuracy'))
            self.model.stop_training = True

In [6]:
# Set all seeds, Control

s=0
w=0
print('** Shuffle Seed:', s)
K.clear_session()
# Set seed values
seed_value = w
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['PYTHONHASHSEED']=str(seed_value)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

trainData, testData = datasets.make_train_data(shuffle_seed=s)
x_predict, y_predict = datasets.make_predict_data(testData)

model = init_all_cnn_c(seed=None)

history = model.fit(
    trainData,
    epochs=3,
    validation_data=testData.prefetch(tf.data.experimental.AUTOTUNE)\
                    .batch(128),
    callbacks=[LR_Callback])

** Shuffle Seed: 0
Making train data...
GCN...
ZCA...
Done!
Making test data...
Done!
Epoch 1/3
391/391 [==============================] - 15s 38ms/step - loss: 2.2993 - accuracy: 0.1040 - val_loss: 2.2817 - val_accuracy: 0.1012 - lr: 0.0100
Epoch 2/3
391/391 [==============================] - 15s 38ms/step - loss: 2.2520 - accuracy: 0.1154 - val_loss: 2.2098 - val_accuracy: 0.1355 - lr: 0.0100
Epoch 3/3
391/391 [==============================] - 15s 38ms/step - loss: 2.2076 - accuracy: 0.1380 - val_loss: 2.2179 - val_accuracy: 0.1340 - lr: 0.0100


In [7]:
# Output should be the same as above

s=0
w=0
print('** Shuffle Seed:', s)
K.clear_session()
# Set seed values
seed_value = w
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['PYTHONHASHSEED']=str(seed_value)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

trainData, testData = datasets.make_train_data(shuffle_seed=s)
x_predict, y_predict = datasets.make_predict_data(testData)

model = init_all_cnn_c(seed=None)

history = model.fit(
    trainData,
    epochs=3,
    validation_data=testData.prefetch(tf.data.experimental.AUTOTUNE)\
                    .batch(128),
    callbacks=[LR_Callback])

** Shuffle Seed: 0
Making train data...
GCN...
ZCA...
Done!
Making test data...
Done!
Epoch 1/3
391/391 [==============================] - 15s 39ms/step - loss: 2.2993 - accuracy: 0.1040 - val_loss: 2.2817 - val_accuracy: 0.1012 - lr: 0.0100
Epoch 2/3
391/391 [==============================] - 15s 38ms/step - loss: 2.2520 - accuracy: 0.1154 - val_loss: 2.2098 - val_accuracy: 0.1355 - lr: 0.0100
Epoch 3/3
391/391 [==============================] - 15s 39ms/step - loss: 2.2076 - accuracy: 0.1380 - val_loss: 2.2179 - val_accuracy: 0.1340 - lr: 0.0100


In [8]:
# Testing independence from system pt. 1: system == weight, shuffle held. pt 2: system != weight, shuffle held
# --> somehow different from just not setting weight

s=0
w=0
print('** Shuffle Seed:', s)
K.clear_session()
# Set seed values
seed_value = w
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['PYTHONHASHSEED']=str(seed_value)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

trainData, testData = datasets.make_train_data(shuffle_seed=s)
x_predict, y_predict = datasets.make_predict_data(testData)

model = init_all_cnn_c(seed=w)

history = model.fit(
    trainData,
    epochs=3,
    validation_data=testData.prefetch(tf.data.experimental.AUTOTUNE)\
                    .batch(128),
    callbacks=[LR_Callback])

** Shuffle Seed: 0
Making train data...
GCN...
ZCA...
Done!
Making test data...
Done!
Epoch 1/3
391/391 [==============================] - 15s 39ms/step - loss: 2.3169 - accuracy: 0.1071 - val_loss: 2.3058 - val_accuracy: 0.1057 - lr: 0.0100
Epoch 2/3
391/391 [==============================] - 15s 39ms/step - loss: 2.3255 - accuracy: 0.1004 - val_loss: 2.3267 - val_accuracy: 0.0992 - lr: 0.0100
Epoch 3/3
391/391 [==============================] - 18s 45ms/step - loss: 2.3267 - accuracy: 0.0997 - val_loss: 2.3267 - val_accuracy: 0.0998 - lr: 0.0100


In [9]:
# Testing whether reseeding undoes the walking. If the following 2 are the same then that's an oopsie

s=0
w=0
print('** Shuffle Seed:', s)
K.clear_session()
# Set seed values
seed_value = w
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['PYTHONHASHSEED']=str(seed_value)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

trainData, testData = datasets.make_train_data(shuffle_seed=s)
x_predict, y_predict = datasets.make_predict_data(testData)

model = init_all_cnn_c(seed=w)

print('Training throwaway...')
history = model.fit(
    trainData,
    epochs=3,
    validation_data=testData.prefetch(tf.data.experimental.AUTOTUNE)\
                    .batch(128),
    callbacks=[LR_Callback])

print('Training real...')
trainData, testData = datasets.make_train_data(shuffle_seed=s)
x_predict, y_predict = datasets.make_predict_data(testData)
model = init_all_cnn_c(seed=w)
history = model.fit(
    trainData,
    epochs=3,
    validation_data=testData.prefetch(tf.data.experimental.AUTOTUNE)\
                    .batch(128),
    callbacks=[LR_Callback])

** Shuffle Seed: 0
Making train data...
GCN...
ZCA...
Done!
Making test data...
Done!
Training throwaway...
Epoch 1/3
391/391 [==============================] - 15s 39ms/step - loss: 2.3169 - accuracy: 0.1071 - val_loss: 2.3058 - val_accuracy: 0.1057 - lr: 0.0100
Epoch 2/3
391/391 [==============================] - 15s 38ms/step - loss: 2.3255 - accuracy: 0.1004 - val_loss: 2.3267 - val_accuracy: 0.0992 - lr: 0.0100
Epoch 3/3
391/391 [==============================] - 16s 40ms/step - loss: 2.3267 - accuracy: 0.0997 - val_loss: 2.3267 - val_accuracy: 0.0998 - lr: 0.0100
Training real...
Making train data...
GCN...
ZCA...
Done!
Making test data...
Done!
Epoch 1/3
391/391 [==============================] - 15s 39ms/step - loss: 2.3169 - accuracy: 0.1071 - val_loss: 2.3058 - val_accuracy: 0.1057 - lr: 0.0100
Epoch 2/3
391/391 [==============================] - 15s 39ms/step - loss: 2.3255 - accuracy: 0.1004 - val_loss: 2.3267 - val_accuracy: 0.0992 - lr: 0.0100
Epoch 3/3
391/391 [=========

In [2]:
# Testing whether the system seed affects weight + shuffle. This cell and the next should come out the same

s=0
w=0
print('** Shuffle Seed:', s)
K.clear_session()
# Set seed values
seed_value = w
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['PYTHONHASHSEED']=str(seed_value)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

print('Training real...')
trainData, testData = datasets.make_train_data(shuffle_seed=17)
x_predict, y_predict = datasets.make_predict_data(testData)
model = init_all_cnn_c(seed=10)
history = model.fit(
    trainData,
    epochs=3,
    validation_data=testData.prefetch(tf.data.experimental.AUTOTUNE)\
                    .batch(128),
    callbacks=[LR_Callback])

** Shuffle Seed: 0
Training real...
Making train data...
GCN...
ZCA...
Done!
Making test data...
Done!
Epoch 1/3
391/391 [==============================] - 17s 44ms/step - loss: 2.3001 - accuracy: 0.1213 - val_loss: 2.2741 - val_accuracy: 0.1361 - lr: 0.0100
Epoch 2/3
391/391 [==============================] - 16s 42ms/step - loss: 2.2181 - accuracy: 0.1569 - val_loss: 2.1473 - val_accuracy: 0.2037 - lr: 0.0100
Epoch 3/3
391/391 [==============================] - 16s 40ms/step - loss: 2.0999 - accuracy: 0.2202 - val_loss: 2.0598 - val_accuracy: 0.2527 - lr: 0.0100


In [3]:
s=123
w=28457
print('** Shuffle Seed:', s)
K.clear_session()
# Set seed values
seed_value = w
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['PYTHONHASHSEED']=str(seed_value)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

print('Training real...')
trainData, testData = datasets.make_train_data(shuffle_seed=17)
x_predict, y_predict = datasets.make_predict_data(testData)
model = init_all_cnn_c(seed=10)
history = model.fit(
    trainData,
    epochs=3,
    validation_data=testData.prefetch(tf.data.experimental.AUTOTUNE)\
                    .batch(128),
    callbacks=[LR_Callback])

** Shuffle Seed: 123
Training real...
Making train data...
GCN...
ZCA...
Done!
Making test data...
Done!
Epoch 1/3
391/391 [==============================] - 16s 40ms/step - loss: 2.3001 - accuracy: 0.1213 - val_loss: 2.2741 - val_accuracy: 0.1361 - lr: 0.0100
Epoch 2/3
391/391 [==============================] - 17s 44ms/step - loss: 2.2181 - accuracy: 0.1569 - val_loss: 2.1473 - val_accuracy: 0.2037 - lr: 0.0100
Epoch 3/3
391/391 [==============================] - 15s 38ms/step - loss: 2.0999 - accuracy: 0.2202 - val_loss: 2.0598 - val_accuracy: 0.2527 - lr: 0.0100


Ok so system seed doesn't impact shuffle or weight if explicitly set

In [4]:
# Test that the seed really walks.
# These 2 outputs should be different.

s=0
w=0
print('** Shuffle Seed:', s)
K.clear_session()
# Set seed values
seed_value = w
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['PYTHONHASHSEED']=str(seed_value)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

trainData, testData = datasets.make_train_data(shuffle_seed=s)
x_predict, y_predict = datasets.make_predict_data(testData)

model = init_all_cnn_c(seed=None)

print('Training throwaway...')
history = model.fit(
    trainData,
    epochs=3,
    validation_data=testData.prefetch(tf.data.experimental.AUTOTUNE)\
                    .batch(128),
    callbacks=[LR_Callback])

print('Training real...')
trainData, testData = datasets.make_train_data(shuffle_seed=s)
x_predict, y_predict = datasets.make_predict_data(testData)
model = init_all_cnn_c(seed=None)
history = model.fit(
    trainData,
    epochs=3,
    validation_data=testData.prefetch(tf.data.experimental.AUTOTUNE)\
                    .batch(128),
    callbacks=[LR_Callback])

** Shuffle Seed: 0
Making train data...
GCN...
ZCA...
Done!
Making test data...
Done!
Training throwaway...
Epoch 1/3
391/391 [==============================] - 16s 40ms/step - loss: 2.2993 - accuracy: 0.1040 - val_loss: 2.2817 - val_accuracy: 0.1012 - lr: 0.0100
Epoch 2/3
391/391 [==============================] - 15s 39ms/step - loss: 2.2520 - accuracy: 0.1154 - val_loss: 2.2098 - val_accuracy: 0.1355 - lr: 0.0100
Epoch 3/3
391/391 [==============================] - 15s 39ms/step - loss: 2.2076 - accuracy: 0.1380 - val_loss: 2.2179 - val_accuracy: 0.1340 - lr: 0.0100
Training real...
Making train data...
GCN...
ZCA...
Done!
Making test data...
Done!
Epoch 1/3
391/391 [==============================] - 16s 40ms/step - loss: 2.2993 - accuracy: 0.1040 - val_loss: 2.2817 - val_accuracy: 0.1012 - lr: 0.0100
Epoch 2/3
391/391 [==============================] - 16s 40ms/step - loss: 2.2520 - accuracy: 0.1154 - val_loss: 2.2098 - val_accuracy: 0.1355 - lr: 0.0100
Epoch 3/3
391/391 [=========

Looks like it won't walk. We need another workaround...like randomly generating the random seed... Let's test

In [7]:
# These should come back the same when run multiple times
seed_value = 0
random.seed(seed_value)
x = random.randint(-10000, 10000)
print(x)
x = random.randint(-10000, 10000)
print(x)
x = random.randint(-10000, 10000)
print(x)

2623
3781
-8674


In [23]:
# Testing method for randomly generating random seed
seed_value = 5
random.seed(seed_value)
for i in range(1001):
    random.randint(-10000, 10000)
x = random.randint(-10000, 10000)
x

-478

##### 

In [24]:
# Final check for shuffle coming back different for multiple runs. These 2 should come back different

s=0
w=0
print('** Shuffle Seed:', s)
K.clear_session()
# Set seed values
seed_value = w
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['PYTHONHASHSEED']=str(seed_value)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

trainData, testData = datasets.make_train_data(shuffle_seed=s)
x_predict, y_predict = datasets.make_predict_data(testData)

model = init_all_cnn_c(seed=w)

print('Training throwaway...')
history = model.fit(
    trainData,
    epochs=3,
    validation_data=testData.prefetch(tf.data.experimental.AUTOTUNE)\
                    .batch(128),
    callbacks=[LR_Callback])

print('Training real...')
model = init_all_cnn_c(seed=w)
history = model.fit(
    trainData,
    epochs=3,
    validation_data=testData.prefetch(tf.data.experimental.AUTOTUNE)\
                    .batch(128),
    callbacks=[LR_Callback])

** Shuffle Seed: 0
Making train data...
GCN...
ZCA...
Done!
Making test data...
Done!
Training throwaway...
Epoch 1/3
391/391 [==============================] - 16s 40ms/step - loss: 2.3169 - accuracy: 0.1071 - val_loss: 2.3058 - val_accuracy: 0.1057 - lr: 0.0100
Epoch 2/3
391/391 [==============================] - 15s 38ms/step - loss: 2.3255 - accuracy: 0.1004 - val_loss: 2.3267 - val_accuracy: 0.0992 - lr: 0.0100
Epoch 3/3
391/391 [==============================] - 15s 38ms/step - loss: 2.3267 - accuracy: 0.0997 - val_loss: 2.3267 - val_accuracy: 0.0998 - lr: 0.0100
Training real...
Epoch 1/3
391/391 [==============================] - 15s 40ms/step - loss: 2.3287 - accuracy: 0.0965 - val_loss: 2.3267 - val_accuracy: 0.0986 - lr: 0.0100
Epoch 2/3
391/391 [==============================] - 15s 39ms/step - loss: 2.3267 - accuracy: 0.0991 - val_loss: 2.3267 - val_accuracy: 0.0992 - lr: 0.0100
Epoch 3/3
391/391 [==============================] - 15s 39ms/step - loss: 2.3267 - accuracy: 0